In [94]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [95]:
PLR_gdf = gpd.read_file('../data/input/berlin/lor_planungsraeume_2021.geojson') # import Planungsräume (PLR) geo data
BZR_gdf = gpd.read_file('../data/input/berlin/lor_bezirksregionen_2021.geojson') # import Bezirksregionen (BZR) geo data
PGR_gdf = gpd.read_file('../data/input/berlin/lor_prognoseraeume_2021.geojson') # import Prognoseräume (PGR) geo data
STREETS_gdf = gpd.read_file('../data/input/berlin/Detailnetz-Strassenabschnitte.geojson') # import Straßen geo data

In [96]:
PLR_gdf.head(5)

,PLR_ID,PLR_NAME,BEZ,STAND,GROESSE_M2,geometry
0,11501341,Karlshorst Süd,11,01.01.2021,2294021.723,"MULTIPOLYGON (((400826.929 5814871.126, 400825..."
1,07601340,Tirschenreuther Ring Ost,07,01.01.2021,413969.326,"MULTIPOLYGON (((387795.701 5807587.153, 387789..."
2,02500831,Wismarplatz,02,01.01.2021,248988.291,"MULTIPOLYGON (((396010.552 5819339.570, 396021..."
3,12601134,Märkisches Zentrum,12,01.01.2021,1127855.841,"MULTIPOLYGON (((387367.428 5828619.763, 387315..."
4,07601547,Horstwalder Straße,07,01.01.2021,941433.954,"MULTIPOLYGON (((391807.174 5805331.900, 391808..."


In [97]:
accident_df = pd.read_csv('../data/temp/raw_accident_dataset_2018-2021.csv')

C:\Users\Domen\AppData\Local\Temp\ipykernel_39656\767954756.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  accident_df = pd.read_csv('../data/temp/raw_accident_dataset_2018-2021.csv')


In [98]:
# accident_df.head(5).T

In [99]:
accident_df = accident_df.drop(columns=["lor", "linrefx", "linrefy"])

In [100]:
accident_gdf = gpd.GeoDataFrame(accident_df, geometry=gpd.points_from_xy(accident_df.longitude, accident_df.latitude),
    crs="EPSG:4326") # transform to GeoDataFrame

### Add PLR Area

In [101]:
accident_gdf = accident_gdf.to_crs(PLR_gdf.crs) # same crs
joined_gdf = gpd.sjoin(accident_gdf, PLR_gdf, op='within', how='left') # join

C:\Users\Domen\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [102]:
accident_df['PLR_ID'] = joined_gdf['PLR_ID'] # add to main df
accident_df['PLR_NAME'] = joined_gdf['PLR_NAME'] # add to main df


### Add BZR area

In [103]:
accident_gdf = accident_gdf.to_crs(BZR_gdf.crs) # same crs
joined_gdf = gpd.sjoin(accident_gdf, BZR_gdf, predicate='within', how='left') # join

C:\Users\Domen\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [104]:
accident_df['BZR_ID'] = joined_gdf['BZR_ID'] # add to main df
accident_df['BZR_NAME'] = joined_gdf['BZR_NAME'] # add to main df


#### Add PGR area

In [105]:
accident_gdf = accident_gdf.to_crs(PGR_gdf.crs) # same crs
joined_gdf = gpd.sjoin(accident_gdf, PGR_gdf, predicate='within', how='left') # join

C:\Users\Domen\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [106]:
accident_df['PGR_ID'] = joined_gdf['PGR_ID'] # add to main df
accident_df['PGR_NAME'] = joined_gdf['PGR_NAME'] # add to main df

In [107]:
joined_gdf

,objectid,bez,strasse,lor_ab_2021,ujahr,umonat,ustunde,uwochentag,ukategorie,uart,...,ustrzustand,longitude,latitude,geometry,index_right,PGR_ID,PGR_NAME,BEZ,STAND,GROESSE_M2
0,112695,2,Samariterviertel,2500729,2018,1,15,4,3,6,...,1,13.475018,52.513597,POINT (396514.624 5819257.316),16.0,0250,Friedrichshain Ost,02,01.01.2021,5.371447e+06
1,112705,12,Ziekowstraße/Freie Scholle,12500824,2018,1,11,2,3,2,...,0,13.291022,52.587259,POINT (384223.627 5827729.725),53.0,1250,Wittenau/Borsigwalde,12,01.01.2021,1.587637e+07
2,112726,2,Barnimkiez,2400520,2018,1,9,3,3,6,...,0,13.420578,52.526019,POINT (392850.817 5820718.312),26.0,0240,Friedrichshain West,02,01.01.2021,4.578339e+06
3,112737,7,Volkspark (Rudolf-Wilde-Park),7200308,2018,1,17,2,3,6,...,0,13.348288,52.481844,POINT (387834.522 5815915.019),32.0,0720,Schöneberg Süd,07,01.01.2021,3.358749e+06
4,112747,3,Niederschönhausen,3200206,2018,1,15,4,3,6,...,1,13.403228,52.583472,POINT (391815.267 5827134.254),52.0,0320,Nördliches Pankow,03,01.01.2021,2.613445e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50114,112174,4,0,4300414,2021,3,16,4,3,6,...,1,13.296345,52.511008,POINT (384383.944 5819240.530),17.0,0430,Charlottenburg Zentrum,04,01.01.2021,9.292147e+06
50115,111541,4,0,4400726,2021,1,1,6,3,9,...,0,13.289496,52.493813,POINT (383873.882 5817339.107),6.0,0440,Wilmersdorf Süd,04,01.01.2021,1.166820e+07
50116,110370,3,0,2400520,2021,1,20,1,3,3,...,1,13.423095,52.527534,POINT (393025.206 5820883.134),26.0,0240,Friedrichshain West,02,01.01.2021,4.578339e+06
50117,109998,7,0,7300619,2021,1,14,7,3,2,...,1,13.359259,52.474367,POINT (388560.601 5815066.337),5.0,0730,Friedenau,07,01.01.2021,4.603888e+06


In [108]:
accident_df

,objectid,bez,strasse,lor_ab_2021,ujahr,umonat,ustunde,uwochentag,ukategorie,uart,...,istsonstige,ustrzustand,longitude,latitude,PLR_ID,PLR_NAME,BZR_ID,BZR_NAME,PGR_ID,PGR_NAME
0,112695,2,Samariterviertel,2500729,2018,1,15,4,3,6,...,0,1,13.475018,52.513597,02500729,Pettenkofer Straße,025007,Frankfurter Allee Nord,0250,Friedrichshain Ost
1,112705,12,Ziekowstraße/Freie Scholle,12500824,2018,1,11,2,3,2,...,0,0,13.291022,52.587259,12500824,Ziekowstraße/Freie Scholle,125008,West 3 - Borsigwalde/Freie Scholle,1250,Wittenau/Borsigwalde
2,112726,2,Barnimkiez,2400520,2018,1,9,3,3,6,...,0,0,13.420578,52.526019,02400520,Barnimkiez,024005,Karl-Marx-Allee Nord,0240,Friedrichshain West
3,112737,7,Volkspark (Rudolf-Wilde-Park),7200308,2018,1,17,2,3,6,...,0,0,13.348288,52.481844,07200308,Volkspark (Rudolph-Wilde-Park),072003,Schöneberg Südwest,0720,Schöneberg Süd
4,112747,3,Niederschönhausen,3200206,2018,1,15,4,3,6,...,0,1,13.403228,52.583472,03200206,Pastor-Niemöller-Platz,032002,Blankenfelde/Niederschönhausen,0320,Nördliches Pankow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50114,112174,4,0,4300414,2021,3,16,4,3,6,...,0,1,13.296345,52.511008,04300414,Schloßstraße,043004,Schloss Charlottenburg,0430,Charlottenburg Zentrum
50115,111541,4,0,4400726,2021,1,1,6,3,9,...,0,0,13.289496,52.493813,04400726,Bismarckallee,044007,Grunewald,0440,Wilmersdorf Süd
50116,110370,3,0,2400520,2021,1,20,1,3,3,...,0,1,13.423095,52.527534,02400520,Barnimkiez,024005,Karl-Marx-Allee Nord,0240,Friedrichshain West
50117,109998,7,0,7300619,2021,1,14,7,3,2,...,0,1,13.359259,52.474367,07300619,Grazer Platz,073006,Friedenau Ost,0730,Friedenau


In [114]:
STREETS_gdf

,element_nr,strassensc,strassenna,str_bez,strassenkl,strassen_1,strassen_2,verkehrsri,bezirk,stadtteil,verkehrseb,beginnt_be,endet_bei_,laenge,gueltig_vo,okstra_id,geometry
0,34610003_34610004.01,00002,Aalemannufer,NaN,IV,G,STRA,B,Spandau,Hakenfelde,0,34610003,34610004,262.5000,2010-01-01,D62521E5E27544729878420C54E6C59C,"MULTILINESTRING ((13.21996 52.57307, 13.22225 ..."
1,40540001_41540003.01,00005,Abbestraße,NaN,V,G,STRA,B,Charlottenburg-Wilmersdorf,Charlottenburg,0,40540001,41540003,182.4500,2010-01-01,275EE05309AF45DCA49E046BBA0CBBCC,"MULTILINESTRING ((13.31987 52.51527, 13.32117 ..."
2,42590002_42590001.01,00022,Afrikanische Straße,NaN,II,G,STRA,B,Mitte,Wedding,0,42590002,42590001,65.3600,2010-01-01,18F55F73EB5346F6A7A719E970B9D4EC,"MULTILINESTRING ((13.33344 52.56096, 13.33332 ..."
3,42590003_42590004.01,00022,Afrikanische Straße,NaN,II,G,STRA,B,Mitte,Wedding,0,42590003,42590004,61.4100,2010-01-01,AE661AB3DB344DD183550F05E15A4BD7,"MULTILINESTRING ((13.33431 52.55736, 13.33458 ..."
4,42590005_42590006.01,00022,Afrikanische Straße,NaN,II,G,STRA,B,Mitte,Wedding,0,42590005,42590006,45.3600,2010-01-01,D47D9FE02F13481FA79966646BA478FA,"MULTILINESTRING ((13.33481 52.55642, 13.33483 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43105,53530049_53530051.01,41423,Irenenstraße,NaN,V,G,STRA,B,Lichtenberg,Rummelsburg,0,53530049,53530051,192.8000,2021-03-04,BBA9B5EF7DB242898DDE1E53868D9227,"MULTILINESTRING ((13.49975 52.50937, 13.50007 ..."
43106,53530051_53530040.01,42456,Rosenfelder Straße,NaN,V,G,STRA,B,Lichtenberg,Rummelsburg,0,53530051,53530040,123.1700,2021-03-04,92AD8579703C4E12B378108ED328B192,"MULTILINESTRING ((13.50249 52.50890, 13.50258 ..."
43107,53530061_53530040.01,40696,Einbecker Straße,NaN,III,G,STRA,B,Lichtenberg,Rummelsburg,0,53530061,53530040,159.5962,2021-03-04,D8CB50FB08E34C0A8AA1A0095A40B0EF,"MULTILINESTRING ((13.50141 52.51033, 13.50335 ..."
43108,53530057_53530061.01,40696,Einbecker Straße,NaN,III,G,STRA,B,Lichtenberg,Rummelsburg,0,53530057,53530061,52.6011,2021-03-04,2EB39A91A5AB43B3B612EC86DE4A3DD6,"MULTILINESTRING ((13.50073 52.51056, 13.50127 ..."


In [118]:
accident_gdf = accident_gdf.to_crs(STREETS_gdf.crs) # same crs
joined_gdf = gpd.sjoin(accident_gdf, STREETS_gdf, predicate='within', how='left')


In [119]:
joined_gdf

,objectid,bez,strasse,lor_ab_2021,ujahr,umonat,ustunde,uwochentag,ukategorie,uart,...,strassen_2,verkehrsri,bezirk,stadtteil,verkehrseb,beginnt_be,endet_bei_,laenge,gueltig_vo,okstra_id
0,112695,2,Samariterviertel,2500729,2018,1,15,4,3,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
1,112705,12,Ziekowstraße/Freie Scholle,12500824,2018,1,11,2,3,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
2,112726,2,Barnimkiez,2400520,2018,1,9,3,3,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
3,112737,7,Volkspark (Rudolf-Wilde-Park),7200308,2018,1,17,2,3,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
4,112747,3,Niederschönhausen,3200206,2018,1,15,4,3,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50114,112174,4,0,4300414,2021,3,16,4,3,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
50115,111541,4,0,4400726,2021,1,1,6,3,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
50116,110370,3,0,2400520,2021,1,20,1,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
50117,109998,7,0,7300619,2021,1,14,7,3,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN


In [126]:
accident_gdf_projected = accident_gdf.to_crs("EPSG:32633")
# 3 meter 
buffered_accidents = accident_gdf_projected.buffer(10) 

In [127]:
# Convert the street segments GeoDataFrame to the same CRS as the buffered points
street_segments_gdf_projected = STREETS_gdf.to_crs("EPSG:32633")

# Perform the spatial join
joined_gdf = gpd.sjoin(accident_gdf_projected, street_segments_gdf_projected, predicate='intersects', how='left')


In [128]:
joined_gdf

,objectid,bez,strasse,lor_ab_2021,ujahr,umonat,ustunde,uwochentag,ukategorie,uart,...,strassen_2,verkehrsri,bezirk,stadtteil,verkehrseb,beginnt_be,endet_bei_,laenge,gueltig_vo,okstra_id
0,112695,2,Samariterviertel,2500729,2018,1,15,4,3,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
1,112705,12,Ziekowstraße/Freie Scholle,12500824,2018,1,11,2,3,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
2,112726,2,Barnimkiez,2400520,2018,1,9,3,3,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
3,112737,7,Volkspark (Rudolf-Wilde-Park),7200308,2018,1,17,2,3,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
4,112747,3,Niederschönhausen,3200206,2018,1,15,4,3,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50114,112174,4,0,4300414,2021,3,16,4,3,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
50115,111541,4,0,4400726,2021,1,1,6,3,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
50116,110370,3,0,2400520,2021,1,20,1,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
50117,109998,7,0,7300619,2021,1,14,7,3,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN


In [111]:
# joined_gdf = joined_gdf.to_crs(streets_gdf.crs)

In [112]:
# joined_gdf = gpd.sjoin(joined_gdf, streets_gdf, op='within', how='left')


In [113]:
# joined_gdf.T